In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
from trulens_eval.keys import *

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


In [7]:
# LLama Index starter example from: https://gpt-index.readthedocs.io/en/latest/getting_started/starter_example.html

# from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, VectorStoreIndex
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import PineconeVectorStore
# from llama_index.indices.vector_store.base import VectorStoreIndex
import pinecone

# Pinecone configuration if using pinecone.
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV  # next to api key in console
)

pinecone_index = pinecone.Index("llm_example")

pinecone_store = PineconeVectorStore(pinecone_index)
index = VectorStoreIndex.from_vector_store(pinecone_store)

#docsearch = Pinecone.from_existing_index(
#    index_name="llmdemo", embedding=embedding
#)

# documents = SimpleDirectoryReader('llama_index/data').load_data()
# index = GPTVectorStoreIndex.from_documents(documents)

# query_engine = index.as_query_engine()
# response = query_engine.query("What did the author do growing up?")
# print(response)

[autoreload of llama_index.vector_stores.types failed: Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    update_instances(old, new)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 303, in update_instances
    ref.__class__

In [6]:
# pip install llama-index --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.8/499.8 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: llama-index
    Found existing installation: llama-index 0.6.19
    Uninstalling llama-index-0.6.19:
      Successfully uninstalled llama-index-0.6.19
Note: you may need to restart the kernel to use updated packages.


In [ ]:
dir(store)

In [ ]:
from trulens_eval import TruLlama, Tru, Query, Feedback
from trulens_eval import tru_feedback

tru = Tru()
Tru().reset_database()

proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)
thread = Tru().start_evaluator(restart=True)

In [ ]:
import numpy as np

# Construct feedback functions.

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.retriever.retrieve.rets[:].node.text
).aggregate(np.min)

feedbacks = [f_lang_match, f_qa_relevance, f_qs_relevance]

l = tru.Llama(engine=query_engine, feedbacks=feedbacks)

In [ ]:
list(l.instrumented())

In [ ]:
res, record = l.query_with_record("What did the author do growing up?")

In [ ]:
record.dict()